<a href="https://colab.research.google.com/github/jordan-dsouza/SlashMarkInternship/blob/main/MNIST_SlashMark/CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow
from tensorflow import keras

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense

class CNN:
    @staticmethod
    def build(width, height, depth, total_classes, Saved_Weights_Path=None):
        #Initialize the Model:
        model = Sequential()

        #First CONV => RELU => POOL Layer:
        model.add(Conv2D(20, 5, 5, border_mode="same", input_shape=(depth, height, width)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering="th"))

        #Second CONV => RELU => POOL Layer:
        model.add(Conv2D(50, 5, 5, border_mode="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering="th"))

        #Third CONV => RELU => POOL Layer:
        #Convolution -> ReLU Activation Function -> Pooling Layer:
        model.add(Conv2D(100, 5, 5, border_mode="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering="th"))

        #FC => RELU layers:
        #Fully Connected Layer -> ReLU Activation Function:
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))

        #Using Softmax Classifier for Linear Classification:
        model.add(Dense(total_classes))
        model.add(Activation("softmax"))

        #If the saved_weights file is already present i.e model is pre-trained, load that weights:
        if Saved_Weights_Path is not None:
            model.load_weights(Saved_Weights_Path)
        return model
# --------------------------------- EOC ------------------------------------


In [ ]:
import numpy as np
import argparse
import cv2
import tensorflow
#from cnn.neural_network import CNN
from keras.utils import np_utils
from keras.optimizers import SGD
from tensorflow import keras
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split


#Parse the Arguments:
ap = argparse.ArgumentParser()
ap.add_argument("-s", "--save_model", type=int, default=-1)
ap.add_argument("-l", "--load_model", type=int, default=-1)
ap.add_argument("-w", "--save_weights", type=str)
args = vars(ap.parse_args())

#Read/Download MNIST Dataset:
print('Loading MNIST Dataset...')
# dataset = fetch_mldata('MNIST Original')
dataset = fetch_openml('mnist_784')

#Read the MNIST data as array of 784 pixels and convert to 28x28 image matrix:
mnist_data = dataset.data.reshape((dataset.data.shape[0], 28, 28))
mnist_data = mnist_data[:, np.newaxis, :, :]

#Divide data into testing and training sets:
train_img, test_img, train_labels, test_labels = train_test_split(mnist_data/255.0, dataset.target.astype("int"), test_size=0.1)

#Now each image rows and columns are of 28x28 matrix type:
img_rows, img_columns = 28, 28

#Transform training and testing data to 10 classes in range [0,classes] ; num. of classes = 0 to 9 = 10 classes:
total_classes = 10			# 0 to 9 labels
train_labels = np_utils.to_categorical(train_labels, 10)
test_labels = np_utils.to_categorical(test_labels, 10)

#Defining and compiling the SGD optimizer and CNN model:
print('\n Compiling model...')
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
clf = CNN.build(width=28, height=28, depth=1, total_classes=10, Saved_Weights_Path=args["save_weights"] if args["load_model"] > 0 else None)
clf.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

#Initially train and test the model; If weight saved already, load the weights using arguments:
b_size = 128		# Batch size
num_epoch = 20		# Number of epochs
verb = 1			# Verbose

#If weights saved and argument load_model; Load the pre-trained model:
if args["load_model"] < 0:
	print('\nTraining the Model...')
	clf.fit(train_img, train_labels, batch_size=b_size, epochs=num_epoch,verbose=verb)

	#Evaluate accuracy and loss function of test data:
	print('Evaluating Accuracy and Loss Function...')
	loss, accuracy = clf.evaluate(test_img, test_labels, batch_size=128, verbose=1)
	print('Accuracy of Model: {:.2f}%'.format(accuracy * 100))


#Save the pre-trained model:
if args["save_model"] > 0:
	print('Saving weights to file...')
	clf.save_weights(args["save_weights"], overwrite=True)


#Show the images using OpenCV and making random selections:
for num in np.random.choice(np.arange(0, len(test_labels)), size=(5,)):
	#Predict the label of digit using CNN:
	probs = clf.predict(test_img[np.newaxis, num])
	prediction = probs.argmax(axis=1)

	#Resize the Image to 100x100 from 28x28 for better view:
	image = (test_img[num][0] * 255).astype("uint8")
	image = cv2.merge([image] * 3)
	image = cv2.resize(image, (100, 100), interpolation=cv2.INTER_LINEAR)
	cv2.putText(image, str(prediction[0]), (5, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

	#Show and print the Actual Image and Predicted Label Value:
	print('Predicted Label: {}, Actual Value: {}'.format(prediction[0],np.argmax(test_labels[num])))

#---------------------- EOC ---------------------


ImportError: cannot import name 'np_utils' from 'keras.utils' (/usr/local/lib/python3.10/dist-packages/keras/utils/__init__.py)

In [ ]:
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import SGD
from keras.utils import to_categorical
from sklearn.datasets import fetch_openml

# Read/Download MNIST Dataset:
print('Loading MNIST Dataset...')
mnist_data = fetch_openml('mnist_784', version=1)
mnist_data.data = mnist_data.data.astype('float32')
mnist_data.data /= 255

# Divide data into testing and training sets:
train_img, test_img, train_labels, test_labels = train_test_split(
    mnist_data.data, mnist_data.target, test_size=0.1)
# Convert DataFrame to numpy array
train_img = train_img.to_numpy()
test_img = test_img.to_numpy()

# Reshape data to match the input shape expected by Keras
train_img = train_img.reshape(train_img.shape[0], 28, 28, 1)
test_img = test_img.reshape(test_img.shape[0], 28, 28, 1)

# Transform training and testing labels to categorical
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Define and compile the SGD optimizer and CNN model:
print('Compiling model...')
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

# Train the model:
print('Training the Model...')
model.fit(train_img, train_labels, batch_size=128, epochs=20, verbose=1)

# Evaluate accuracy and loss function on test data:
print('Evaluating Accuracy and Loss Function...')
loss, accuracy = model.evaluate(test_img, test_labels, batch_size=128, verbose=1)
print('Accuracy of Model: {:.2f}%'.format(accuracy * 100))

# Save the model:
print('Saving model to file...')
model.save("mnist_model.h5")

# Show the images using OpenCV and making random selections:
for num in np.random.choice(np.arange(0, len(test_labels)), size=(5,)):
    # Predict the label of digit using CNN:
    probs = model.predict(test_img[np.newaxis, num])
    prediction = probs.argmax(axis=1)

    # Resize the Image to 100x100 for better view:
    image = (test_img[num][:, :, 0] * 255).astype("uint8")
    image = cv2.merge([image] * 3)
    image = cv2.resize(image, (100, 100), interpolation=cv2.INTER_LINEAR)
    cv2.putText(image, str(prediction[0]), (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

    # Show and print the Actual Image and Predicted Label Value:
    print('Predicted Label: {}, Actual Value: {}'.format(prediction[0], np.argmax(test_labels[num])))
    cv2.imshow("Digit", image)
    cv2.waitKey(0)

cv2.destroyAllWindows()


Loading MNIST Dataset...


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Compiling model...
Training the Model...
Epoch 1/20
493/493 [==============================] - 28s 54ms/step - loss: 0.3553 - accuracy: 0.8960
Epoch 2/20
493/493 [==============================] - 26s 53ms/step - loss: 0.1469 - accuracy: 0.9561
Epoch 3/20
493/493 [==============================] - 26s 53ms/step - loss: 0.1039 - accuracy: 0.9685
Epoch 4/20
493/493 [==============================] - 27s 56ms/step - loss: 0.0816 - accuracy: 0.9750
Epoch 5/20
493/493 [==============================] - 26s 53ms/step - loss: 0.0663 - accuracy: 0.9794
Epoch 6/20
493/493 [==============================] - 26s 54ms/step - loss: 0.0560 - accuracy: 0.9836
Epoch 7/20
493/493 [==============================] - 26s 53ms/step - loss: 0.0471 - accuracy: 0.9857
Epoch 8/20
493/493 [==============================] - 27s 54ms/step - loss: 0.0408 - accuracy: 0.9880
Epoch 9/20
493/493 [==============================] - 27s 54ms/step - loss: 0.0364 - accuracy: 0.9888
Epoch 10/20
493/493 [====================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Predicted Label: 9, Actual Value: 9


DisabledFunctionError: cv2.imshow() is disabled in Colab, because it causes Jupyter sessions
to crash; see https://github.com/jupyter/notebook/issues/3935.
As a substitution, consider using
  from google.colab.patches import cv2_imshow
